In [ ]:
#===========================================================================================
# ElasticSearch 텍스트 임베딩 테스트 예제
# - 문장들을  추출 요약해서 요약문장을 만들고, 요약 문장의 평균을 구하여 문장 embedding을 생성하여 ES에 인덱스에 vector 추가하고, 검색하는 예제임
# 
# -여기서는 elasticsearch 7.17.3 때를 기준으로 설명함.
# -** 따라서 elasticsearch python 모듈도 7.17.3 을 설치해야 함
# - elasticsearch 모듈 8.x 부터는 구문의 많이 변경되었음.
# - 예 : index 생성:  body로 모든 변수들를 지정하는 데시, 명시적으로 모든 변수들을 최상으로 지정해 줘야함.
# => 참고: https://towardsdatascience.com/important-syntax-updates-of-elasticsearch-8-in-python-4423c5938b17   

# =>ElasticSearch 7.3.0 버전부터는 cosine similarity 검색을 지원한다.
# => 데이터로 고차원벡터를 집어넣고, 벡터형식의 데이터를 쿼리(검색어)로 하여 코사인 유사도를 측정하여 가장 유사한 데이터를 찾는다.
# => 여기서는 ElasticSearch와 S-BERT를 이용함
# => ElasticSearch에 index 파일은 index_1.json /데이터 파일은 KorQuAD_v1.0_train_convert.json 참조
#
# => 참고자료 : https://skagh.tistory.com/32
#===========================================================================================

# sentenceTransformers 라이브러리 설치
#!pip install -U sentence-transformers

# elasticsearch 서버 접속 모듈 설치
# !pip install elasticsearch==7.17

# 한국어 문장 분리기(kss) 설치
#!pip install kss

# 추출 요약 설치
#!pip install bert-extractive-summarizer

In [ ]:
import torch
from sentence_transformers import SentenceTransformer, util
from sentence_transformers.cross_encoder import CrossEncoder

import kss
import numpy as np
import json
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk
from tqdm.notebook import tqdm

from elasticsearch import Elasticsearch
from elasticsearch import helpers

# 추출 요약
from summarizer.sbert import SBertSummarizer

# FutureWarning 제거
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning) 

import sys
sys.path.append('..')
from myutils import seed_everything, GPU_info
device = GPU_info()
#device = torch.device('cpu')

seed_everything(111)

# elastic 서버 접속 테스트
#es = Elasticsearch("https://192.168.0.91:9200/", verify_certs=False)
#es = Elasticsearch("http://192.168.0.130:9200/")
#es.info()


In [ ]:
# s-bert 모델 테스트
#sbert_model_path = '../../../model/sbert/klue-sbert-v1.0'
sbert_model_path = 'bongsoo/albert-small-kor-sbert-v1'

#=====================================================================
# 임베딩 모델 설정
# - cpu 모델로 실행할때는 device=cpu, 기본은 GPU임
embedder = SentenceTransformer(sbert_model_path, device=device)

text = '나는 오늘 밥을 먹는다.'
vectors = embedder.encode(text, convert_to_tensor=True)
print(f'vector_len:{len(vectors)}')
#=====================================================================
# 추출요약 모델 설정
summarizer_sbert_model_path = '../../../model/sbert/sbert-albert-small-nli-cls-64-sts-128ss'
summarizer_model = SBertSummarizer(sbert_model_path)
print(summarizer_model)
#=====================================================================
# crossencoder 모델 설정
crossencoder_model_path = 'bongsoo/albert-small-kor-sbert-v1'
crossencoder = CrossEncoder(crossencoder_model_path, device=device)
print(crossencoder)
#=====================================================================

In [ ]:
# 요약 테스트 

paragraph = '''
1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로 해서 하나의 교향곡을 쓰려는 뜻을 갖는다. 
이 시기 바그너는 1838년에 빛 독촉으로 산전수전을 다 걲은 상황이라 좌절과 실망에 가득했으며 메피스토펠레스를 만나는 파우스트의 심경에 공감했다고 한다. 또한 파리에서 아브네크의 지휘로 파리 음악원 관현악단이 연주하는 베토벤의 교향곡 9번을 듣고 깊은 감명을 받았는데, 이것이 이듬해 1월에 파우스트의 서곡으로 쓰여진 이 작품에 조금이라도 영향을 끼쳤으리라는 것은 의심할 여지가 없다. 
여기의 라단조 조성의 경우에도 그의 전기에 적혀 있는 것처럼 단순한 정신적 피로나 실의가 반영된 것이 아니라 베토벤의 합창교향곡 조성의 영향을 받은 것을 볼 수 있다. 
그렇게 교향곡 작곡을 1839년부터 40년에 걸쳐 파리에서 착수했으나 1악장을 쓴 뒤에 중단했다. 
또한 작품의 완성과 동시에 그는 이 서곡(1악장)을 파리 음악원의 연주회에서 연주할 파트보까지 준비하였으나, 실제로는 이루어지지는 않았다. 
결국 초연은 4년 반이 지난 후에 드레스덴에서 연주되었고 재연도 이루어졌지만, 이후에 그대로 방치되고 말았다. 
그 사이에 그는 리엔치와 방황하는 네덜란드인을 완성하고 탄호이저에도 착수하는 등 분주한 시간을 보냈는데, 그런 바쁜 생활이 이 곡을 잊게 한 것이 아닌가 하는 의견도 있다. 
한편 1840년부터 바그너와 알고 지내던 리스트가 잊혀져 있던 1악장을 부활시켜 1852년에 바이마르에서 연주했다. 
이것을 계기로 바그너도 이 작품에 다시 관심을 갖게 되었고, 그 해 9월에는 총보의 반환을 요구하여 이를 서곡으로 간추린 다음 수정을 했고 브라이트코프흐 & 헤르텔 출판사에서 출판할 개정판도 준비했다. 
1853년 5월에는 리스트가 이 작품이 수정되었다는 것을 인정했지만, 끝내 바그너의 출판 계획은 무산되고 말았다. 
이후 1855년에 리스트가 자신의 작품 파우스트 교향곡을 거의 완성하여 그 사실을 바그너에게 알렸고, 바그너는 다시 개정된 총보를 리스트에게 보내고 브라이트코프흐 & 헤르텔 출판사에는 20루이의 금을 받고 팔았다. 
또한 그의 작품을 “하나하나의 음표가 시인의 피로 쓰여졌다”며 극찬했던 한스 폰 뷜로가 그것을 피아노 독주용으로 편곡했는데, 리스트는 그것을 약간 변형되었을 뿐이라고 지적했다. 
이 서곡의 총보 첫머리에는 파우스트 1부의 내용 중 한 구절을 인용하고 있다. 
이 작품은 라단조, Sehr gehalten(아주 신중하게), 4/4박자의 부드러운 서주로 서주로 시작되는데, 여기에는 주요 주제, 동기의 대부분이 암시, 예고되어 있다. 
첫 부분의 저음 주제는 주요 주제(고뇌와 갈망 동기, 청춘의 사랑 동기)를 암시하고 있으며, 제1바이올린으로 더욱 명확하게 나타난다. 
또한 그것을 이어받는 동기도 중요한 역할을 한다. 
여기에 새로운 소재가 더해진 뒤에 새로운 주제도 연주된다. 
주요부는 Sehr bewegt(아주 격동적으로), 2/2박자의 자유로운 소나타 형식으로 매우 드라마틱한 구상과 유기적인 구성을 하고 있다. 
여기에는 지금까지의 주제나 소재 외에도 오보에에 의한 선율과 제2주제를 떠올리게 하는 부차적인 주제가 더해지는데, 중간부에서는 약보3이 중심이 되고 제2주제는 축소된 재현부에서 D장조로 재현된다. 
마지막에는 주요 주제를 회상하면서 조용히 마친다.
'''


import time

start_time = time.time()
    
paragraph_summarize = summarizer_model(paragraph, 
                                        min_length=10, 
                                        num_sentences=3)
end_time = time.time() - start_time

print(f'time:{end_time}')
print(f'summarize:{paragraph_summarize}')


In [ ]:
# 인덱싱 함수 
def index_data():
    es.indices.delete(index=INDEX_NAME, ignore=[404])
    
    count = 0
       
    # 인덱스 생성
    with open(INDEX_FILE) as index_file:
        source = index_file.read().strip()
      
        count += 1
        print(f'{count}:{source}')
      
        es.indices.create(index=INDEX_NAME, body=source)
        
    count = 0
    
    # DATA 추기
    with open(DATA_FILE) as data_file:
        for line in data_file:
            line = line.strip()
            
            json_data = json.loads(line)
            docs = []
            
            for data in tqdm(json_data):
                count += 1
                doc = {} # dict로 선언
                
                paragraph = data['paragraph']
                
                #================================================================
                # 추출요약 적용된 경우에는 입력문장에 대한 요약문장 추출함.
                if SUMMARIZER == True:
                    paragraph_summarize = summarizer_model(paragraph, 
                                                           min_length=MIN_LENGTH, 
                                                           num_sentences=SUMMARIZER_NUM_SENTENCE)
                    # 요약문이 있으면 제목. + 요약문  담고, 없으면 제목. + 원본문장 담음.
                    if paragraph_summarize:
                        doc['summarize'] = paragraph_summarize
                    else:
                        doc['summarize'] = paragraph
                #================================================================
                
                doc['title'] = data['title']            # 제목 설정
                doc['paragraph'] = data['paragraph']    # 문장 담음.
                
                docs.append(doc)
                
                if count % BATCH_SIZE == 0:
                    index_batch(docs)
                    docs = []
                    print("Indexed {} documents.".format(count))
                  
                # ** 10 개만 보냄
                #if count >= 10:
                #    break
                    
            if docs:
                index_batch(docs)
                print("Indexed {} documents.".format(count))
                    
    es.indices.refresh(index=INDEX_NAME)
    print("=== End Done indexing===")
    
    
# 문단(paragraph)들 분리
# 문장으로 나누고, 해당 vector들의 평균을 구함.
# =>굳이 elasticsearch에 문단 벡터는 추가하지 않고, title 벡터만 이용해도 되므로 주석처리함

def paragraph_index(paragraph):
    avg_paragraph_vec = np.zeros((1,768))
    sent_count = 0
    
    # ** kss로 분할할때 히브리어: מר, 기타 이상한 특수문자 있으면 에러남. 
    # 따라서 여기서는 그냥 . 기준으로 문장을 나누고 평균을 구함
    # 하나의 문장을 읽어와서 .기준으로 나눈다.
    sentences = [sentence for sentence in paragraph.split('. ') if sentence != '']
    for sent in sentences:
        # 문장으로 나누고, 해당 vector들의 평균을 구함.
        avg_paragraph_vec += embed_text([sent])
        sent_count += 1
        
    '''
    # kss로 분할할때 줄바꿈 있으면, 파싱하는데 에러남.따라서 "\n"는 제거함
    paragraph = paragraph.replace("\n","")
    
    print("==Start paragraph_index==")
    print(paragraph)
    for sent in kss.split_sentences(paragraph):
        # 문장으로 나누고, 해당 vector들의 평균을 구함.
        avg_paragraph_vec += embed_text([sent])
        sent_count += 1
    '''
    
    avg_paragraph_vec /= sent_count
    return avg_paragraph_vec.ravel(order='C') # 1차원 배열로 변경


def index_batch(docs):
   
    # 제목 벡터를 구함
    titles = [doc["title"] for doc in docs]  
    title_vectors = embed_text(titles)      
    
    # 원본문장 벡터를 구함 => 전체벡터를 구함
    paragraphs = [doc["paragraph"] for doc in docs]   
     # * cpu로 문장별 평균을 구하는 경우에는 임베딩하는데 너무 오래 걸리므로 주석처리함
    #paragraph_vectors = [paragraph_index(doc["paragraph"]) for doc in tqdm(docs)]
    paragraph_vectors = embed_text(paragraphs)
      
    #=========================================================================
    # 요약문 각 문장별 벡터를 구하고 평균을 냄
    if SUMMARIZE_AVG_INDEX == True:
        summarize_vectors = [paragraph_index(doc["summarize"]) for doc in docs]
    # 요약문 전체 벡터를 구함    
    else:
        paragraph_summarizes = [doc["summarize"] for doc in docs]
        summarize_vectors = embed_text(paragraph_summarizes)
     #=========================================================================    
   
    requests = []
    
    # ES request 할 리스트 정의
    for i, doc in enumerate(tqdm(docs)):
        request = {}  #dict 정의
        
        # 요약벡터 = 타이틀벡터 + 요약벡터 의 평균으로 함
        summarize_vector = (title_vectors[i] + summarize_vectors[i]) / 2
        
        request["title"] = doc["title"]            # 제목               
        request["paragraph"] = doc["paragraph"]    # 문장
        request["summarize"] = doc["summarize"]    # 요약문
        
        request["_op_type"] = "index"        
        request["_index"] = INDEX_NAME
        
        request["title_vector"] = title_vectors[i]          # 제목 벡터
        request["paragraph_vector"] = paragraph_vectors[i]  # 문장 벡터
        request["summarize_vector"] = summarize_vector #summarize_vectors[i]  # 요약문 벡터
        
        requests.append(request)
        
    # batch 단위로 한꺼번에 es에 데이터 insert 시킴     
    bulk(es, requests)
    
# embedding 모델에서 vector를 구함    
def embed_text(input):
    vectors =  embedder.encode(input, convert_to_tensor=True)
    return [vector.numpy().tolist() for vector in vectors]
          

In [ ]:
#======================================================================================
# ElasticSearch(이하:ES) 데이터 인텍싱
# - ElasticSearch(이하:ES)에 KorQuAD_v1.0_train_convert.json 파일에 vector값을 구하여 index 함
#
# => index 명 : korquad
# => index 구조 : index_1.json 파일 참조
# => BATCH_SIZE : 100 => 100개의 vector값을 구하여, 한꺼번에 ES에 인텍스 데이터를 추가함.
#======================================================================================
INDEX_NAME = 'korquad-albert-small-kor-sbert-v1'      # ES 인덱스 명 (*소문자로만 지정해야 함)
INDEX_FILE = './data/index_summarize.json'            # 인덱스 구조 파일
DATA_FILE = './data/KorQuAD_v1.0_train_convert.json'  # 인덱싱할 파일경로
BATCH_SIZE = 100

SUMMARIZER = True            # TRUE면 추출요약을 한다.
SUMMARIZER_NUM_SENTENCE = 2  # 추출요약할때 몇문장으로 요약할지
SUMMARIZE_AVG_INDEX = True   # True면 요약문장들에 대해 각 문장의 embedding을 구하고, 평균을 낸다.
MIN_LENGTH = 20              # 추출요약할때 해당 길이보다 작은 문장은 제외함.

# 1. elasticsearch 접속
es = Elasticsearch("http://192.168.0.27:9200/")
print(es.info())

# 2. index 처리
index_data()

In [ ]:
# kibana 콘솔창에 접속해서 계수 확인
# http://192.168.0.130:5601/app/dev_tools 에 접속해서 해야함

## 입력 ##
# GET korquad/_count

## 출력 ###
'''
{
  "count" : 1420,
  "_shards" : {
    "total" : 2,
    "successful" : 2,
    "skipped" : 0,
    "failed" : 0
}
'''    

In [ ]:
# 검색 하기

import time
from elasticsearch import Elasticsearch

def run_query_loop():
    while True:
        try:
            handle_query()
        except KeyboardInterrupt:
            return
        
def handle_query():
    
    query = input("검색어 입력: ")
    
    start_embedding_time = time.time()
    query_vector = embed_text([query])[0]
    end_embedding_time = time.time() - start_embedding_time
    
    # 쿼리 구성
    script_query = {
        "script_score":{
            "query":{
                "match_all": {}},
            "script":{
                "source": "cosineSimilarity(params.query_vector, doc['summarize_vector']) + 1.0",  # 뒤에 1.0 은 코사인유사도 측정된 값 + 1.0을 더해준 출력이 나옴(doc['summarize_vector'])
                "params": {"query_vector": query_vector}
            }
        }
    }
    
    #print('query\n')
    #print(script_query)
    
    # 실제 ES로 검색 쿼리 날림
    start_search_time = time.time()
    response = es.search(
        index=INDEX_NAME,
        body={
            "size": SEARCH_SIZE,
            "query": script_query,
            "_source":{"includes": ["title", "summarize"]}
        }
    )
    end_search_time = time.time() - start_search_time
    
    print("{} total hits.".format(response["hits"]["total"]["value"])) 

        
    # 쿼리 응답 결과값에서 _id, _score, _source 등을 뽑아냄
    # print(response)
    summarizes = []
    titles = [] 
    bi_scores = []
    for hit in response["hits"]["hits"]: 
        '''
        print("index:{}, type:{}".format(hit["_index"], hit["_type"]))
        print("id: {}, score: {}".format(hit["_id"], hit["_score"])) 
        
        print(f'[제목] {hit["_source"]["title"]}')
        
        print('[요약문]')
        print(hit["_source"]["summarize"]) 
        print()
                
        '''
        # 리스트에 저장해둠
        titles.append(hit["_source"]["title"])
        summarizes.append(hit["_source"]["summarize"])
        bi_scores.append(hit["_score"])
        
    #========================================================================================================
    # corssencoder 처리 
    # => 위에서 응답받은 문장들을 다시 한번 [query, title] 문장쌍으로 만들어서 스코어를 구한다.
    #========================================================================================================
        
    # crossencoder로 스코어 구하기 위해 [query, title] 쌍으로 문장을 만든다.
    
    start_cross_time = time.time()
    
    sentence_combinations = [[query, hit["_source"]["title"]] for hit in response["hits"]["hits"] if hit != '']
    #print(sentence_combinations)
    #print('\n')
        
    cross_scores = crossencoder.predict(sentence_combinations)+1 
    
    end_cross_time = time.time() - start_cross_time
        
    # 내림 차순으로 정렬
    dec_cross_scores = reversed(np.argsort(cross_scores))
    
    # 내림차순으로 출력
    for idx in dec_cross_scores:
        print("cross:{:.2f}\bi:{:.2f}\t[제목]:{}\n{}\n".format(cross_scores[idx], bi_scores[idx], titles[idx], summarizes[idx]))
    
    # 처리 시간들 출력
    print("embedding time: {:.2f} ms".format(end_embedding_time * 1000)) 
    print("search time: {:.2f} ms".format(end_search_time * 1000)) 
    print("cross time: {:.2f} ms".format(end_cross_time * 1000))
    print('\n')
     #========================================================================================================    
        

In [ ]:
#====================================================================
# ES 인덱싱된 내용 검색 
# => cosineSimilarity 스크립트를 이용하여 ES로 query 함(*이때 SEARCH_SIZE를 몇개로 할지 지정할수 있음)
# => 쿼리 응답 결과 값에서 _id, _score, _source 등을 뽑아냄
#====================================================================

INDEX_NAME = 'korquad-albert-small-kor-sbert-v1' # 요약문 평균값 처리
#INDEX_NAME = 'korquad-klue-sbert-v1.0-noavg' # 요약문 평균값 처리 안한경우

SEARCH_SIZE = 20

# 1. elasticsearch 접속
es = Elasticsearch("http://192.168.0.27:9200/")
print(es.info)

# 2. query 처리
run_query_loop()

In [ ]:
#==============================================================================================
# ES index에 데이터 추가하가
# => 추가할 데이터는 {'paragraph': 내용, 'title': 제목} 기존 입려된 방식대로(사전) 입력 되어야 함
#===============================================================================================

# ES에 이미 생성된 index
INDEX_NAME = 'korquad'
BATCH_SIZE = 30


# 1.추가할 데이터 준비
title = [
    '제주도', 
    '한라산',
    '서울특별시'
        ]

paragraph = [
    '대한민국의 남서쪽에 있는 섬. 행정구역상 광역자치단체인 제주특별자치도의 관할. 한국의 섬 중에서 가장 크고 인구가 많은 섬으로 면적은 1833.2㎢이다. 제주도 다음 2번째 큰 섬인 거제도의 5배 정도 된다. 인구는 약 68만 명.',
    '대한민국에서 가장 큰 섬인 제주도에 있으며 대한민국의 실효지배 영토 내의 최고봉이자 가장 높은 산(해발 1,947m). 대한민국의 국립공원 중 하나이다. 국립공원 전역이 유네스코 세계유산으로 지정되었다.',
    '대한민국의 수도인 서울은 현대적인 고층 빌딩, 첨단 기술의 지하철, 대중문화와 예것이 공존하는 대도시. 주목할 만한 명소로는 초현대적 디자인의 컨벤션 홀인 동대문디자인플라자, 한때 7,000여 칸의 방이 자리하던 경복궁, 조계사가 있다',
            ]

# {'paragraph': "", 'title': ""}

# 2. elasticsearch 접속
es = Elasticsearch("http://192.168.0.27:9200/")
print(es.info)

doc = {}
docs = []
count = 0

# 3. batch 사이즈 만큼식 ES에 추가
# => 추가할 데이터는 {'paragraph': 내용, 'title': 제목} 기존 입려된 방식대로(사전) 입력 되어야 함
for title, paragraph in zip(title, paragraph):
    doc = {}
    doc['paragraph'] = paragraph
    doc['title'] = title
    docs.append(doc)
    count += 1
    if count % BATCH_SIZE == 0:
        index_batch(docs)
        docs = []
        print("Indexed {} documents.".format(count))
   
# docs 이 있으면 전송
if docs:
    index_batch(docs)
    print("Indexed {} documents(end).".format(count))


In [ ]:
#==============================================================================================
# ES 데이터 조회하기
#==============================================================================================
INDEX_NAME = 'korquad'

# 1. elasticsearch 접속
es = Elasticsearch("http://192.168.0.27:9200/")
print(es.info)

###########################################################
# 인덱스내 데이터 조회 => query 이용
###########################################################
def search(index, data=None):
    if data is None: #모든 데이터 조회
        data = {"match_all":{}}
    else:
        data = {"match": data}
        
    body = {"query": data}
    res = es.search(index=index, body=body)
    return res
###########################################################

# 모든 데이터 조회
#sr = search(index=INDEX_NAME)
#pprint.pprint(sr)

# 단일 필드 조회
sr = search(index=INDEX_NAME, data = {'title': '제주도'})
print(sr)


In [ ]:
#==============================================================================================
# ES index에 데이터 삭제하기
#==============================================================================================
INDEX_NAME = 'korquad'

# 1. elasticsearch 접속
es = Elasticsearch("http://192.168.0.27:9200/")
print(es.info)

############################################################
## 1: 인덱스 내의 데이터 삭제 => query 이용
############################################################
def delete(index, data):
    if data is None:  # data가 없으면 모두 삭제
        data = {"match_all":{}}
    else:
        data = {"match": data}
        
    body = {"query": data}
    return es.delete_by_query(index, body=body)

############################################################
## 2: 인덱스 내의 데이터 삭제 => id 이용
############################################################
def delete_by_id(index, id):
    return es.delete(index, id=id)

############################################################
## 3: 인덱스 자체 삭제
############################################################
def delete_index(index):
    if es.indices.exists(index=index):
        return es.indices.delete(index=index)


# 1: query 이용 데이터 삭제
delete(index=INDEX_NAME, data={'title':'한라산'})

# 3: 인덱스 자체 삭제
#delete_index(index=INDEX_NAME)


In [ ]:
#==============================================================================================
# ES index에 데이터 업데이트하기
#==============================================================================================
INDEX_NAME = 'korquad'

# 1. elasticsearch 접속
es = Elasticsearch("http://192.168.0.27:9200/")
print(es.info)

############################################################
## 1: 인덱스 내의 데이터 업데이트=>_id 에 데이터 업데이트
############################################################
def update(index, id, doc, doc_type):
    
    body = {
        'doc': doc
    }
    
    res=es.update(index=index, id=id, body=body, doc_type=doc_type)
    return res
############################################################

#=====================================================================
# 검색해서, _id, _type을 구함
sr = search(index=INDEX_NAME, data = {'title': '제주도'})

print('\n')
print("===[검색 결과]===")
for hits in sr["hits"]["hits"]:
    id = hits["_id"]      # id
    type = hits["_type"]  # type
    
    print(f'id: {id}')
    print(f'type: {type}')
    print(f'title:{hits["_source"]["title"]}')
    print(f'paragraph:{hits["_source"]["paragraph"]}')
    print('\n')
    
    # update 시킴
    print("===[업데이트]===")
    doc = {'paragraph': '제주도는 대한민국에 가장 남쪽에 있는 섬으로, 인구는 약 71만명이며, 화산섬으로 관광자원이 많은 천혜의 관광지 이다.'}
    print(doc)
    print('\n')
    
    ur=update(index=INDEX_NAME, id=id, doc=doc, doc_type=type)
    print("===[업데이트 결과]===")
    print(ur)
    print('\n')

#=====================================================================

# 인덱스 refresh 함
# elasticsearch의 자동 새로고침의 시간은 1초 정도 소요
# 따라서 코드에 아래 명령어를 입력하지 않았을 경우 검색을 하지 못할 가능성도 존재
es.indices.refresh(index=INDEX_NAME)

# 제주도로 검색해서 한번더 확인
sr = search(index=INDEX_NAME, data = {'title': '제주도'})

print("===[재검색 결과]===")
for hits in sr["hits"]["hits"]:
    
    print(f'id:{hits["_id"]}')
    print(f'type: {hits["_type"]}')
    print(f'title:{hits["_source"]["title"]}')
    print(f'paragraph:{hits["_source"]["paragraph"]}')
    
              